In [ ]:

from polygraphy.backend.trt import CreateConfig, Profile, TrtRunner, TacticRecorder, TacticReplayer, TacticReplayData
from polygraphy.backend.trt import network_from_onnx_path, engine_from_network, create_config, save_engine

from polygraphy.backend.onnxrt import OnnxrtRunner, session_from_onnx
from polygraphy.backend.onnx import modify_outputs, onnx_from_path, save_onnx
from polygraphy.backend.onnx.util import str_from_onnx, all_tensor_names

from polygraphy.comparator import Comparator, CompareFunc, DataLoader
from polygraphy.common import TensorMetadata
from polygraphy.json import load_json


ONNX_MODEL = "./models/crnn.onnx"

In [ ]:
# Inspect the onnx model
onnx_proto = onnx_from_path(ONNX_MODEL)
onnx_str = str_from_onnx(onnx_proto, mode='attrs')
print(onnx_str)

In [ ]:
# Modify the onnx model's outputs

modified = modify_outputs(onnx_proto,['646','351'])# mark tensor 646 and 351 as outputs, or you can use constants.MARK_ALL to mark all tensors as outputs
onnx_modified = str_from_onnx(modified, mode='attrs')
print(onnx_modified)
_=save_onnx(modified,"modified.onnx")

In [ ]:
builder, network, parser = network_from_onnx_path(ONNX_MODEL)

In [ ]:
recorder = TacticRecorder("trt_tactics.json")
optimization_profiles = [Profile().add("input_0", min=(1,3,32,1024), opt=(8,3,32,1024), max=(16,3,32,1024))]
trt_config = CreateConfig(max_workspace_size=4096000000,
                          tf32=None,
                          fp16=True,
                          int8=None,
                          profiles=optimization_profiles,
                          calibrator=None,
                          strict_types=None, 
                          load_timing_cache=None,
                          algorithm_selector=recorder,
                          sparse_weights=None,
                          tactic_sources=[0,1,2],
                          restricted=None)

engine = engine_from_network((builder,network), config=trt_config)
save_engine(engine,"crnn_fp16.plan")

# To ensure the reproducibility of the engine build

# replayer = TacticReplayer("trt_tactics.json")
# optimization_profiles = [Profile().add("input_0", min=(1,3,32,1024), opt=(8,3,32,1024), max=(16,3,32,1024))]
# trt_config = CreateConfig(max_workspace_size=2048000000,
#                          tf32=None,
#                          fp16=True,
#                          int8=None,
#                          profiles=optimization_profiles,
#                          calibrator=None,
#                          strict_types=None, 
#                          load_timing_cache=None,
#                          algorithm_selector=replayer,
#                          sparse_weights=None,
#                          tactic_sources=[0,1,2],
#                          restricted=None)

In [ ]:
# Run inference with comparator

import numpy as np
  
meta = TensorMetadata()
meta.add("input_0", np.float32, [16,3,32,1024])
loader = DataLoader(input_metadata=meta)

onnx_session = session_from_onnx(ONNX_MODEL)

runners = [TrtRunner(engine),
           OnnxrtRunner(onnx_session)]

run_results = Comparator.run(runners, data_loader=loader, save_inputs_path="inputs.json")


In [ ]:
# Compare results across runners

compare = CompareFunc.basic_compare_func(check_shapes=True, rtol=10, atol=0.15, fail_fast=None, check_error_stat='max')

accuracy_result = Comparator.compare_accuracy(run_results,compare_func=compare)

In [ ]:
import tensorrt as trt

EXCLUDE_LAYERS = [trt.LayerType.SHAPE,trt.LayerType.CONSTANT,trt.LayerType.CONCATENATION,trt.LayerType.GATHER,trt.LayerType.SLICE,trt.LayerType.SHUFFLE]
num_layer = network.num_layers
for i in range(num_layer-1,255,-1):
    layer = network.get_layer(i)
    if layer.type not in EXCLUDE_LAYERS:
        print("setting layer_{} to fp32".format(i))      
        layer.reset_precision()
        layer.precision = trt.float32
        layer.set_output_type(0,trt.float32)
          
trt_config = CreateConfig(max_workspace_size=4096000000,
                          tf32=None,
                          fp16=True,
                          int8=None,
                          profiles=optimization_profiles,
                          calibrator=None,
                          strict_types=True, 
                          load_timing_cache=None,
                          algorithm_selector=None,
                          sparse_weights=None,
                          tactic_sources=[0,1,2],
                          restricted=None)

engine_v2 = engine_from_network((builder,network), config=trt_config)
save_engine(engine_v2,"crnn_fp16_v2.plan")   

In [ ]:

runners_v2 = [TrtRunner(engine_v2),
              OnnxrtRunner(onnx_session)]

loader = []
for input_data_path in ['inputs.json']:
    loader.extend(load_json(input_data_path, description='input data'))

run_results_v2 = Comparator.run(runners_v2, data_loader=loader)

In [ ]:
accuracy_result = Comparator.compare_accuracy(run_results_v2,compare_func=compare)

In [ ]:
# Validate NaN and inf
Comparator.validate(run_results_v2, check_inf=True, check_nan=True)

In [ ]:
# direct access to the run_result of the TRTrunner
run_results_v2[0][1][0]['output_0']